In [1]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


class CONMEBOL_ESPN:
    def __init__(self, final_date):
        self.games = []
        self.url = "https://www.espn.com.br/futebol/resultados/_/liga/CONMEBOL.LIBERTADORES/data"
        self.__get_valid_dates(final_date)
        
    
    def __get_valid_dates(self, final_date):
        r = requests.get(f"{self.url}/{final_date}")
        soup = BeautifulSoup(r.text)
        
        for script in soup.find_all("script", {"type": None, "src": None}):
            if script.text.startswith("window.espn.scoreboardData"):
                data = json.loads(script.text[30:-337])
                break
        
        intervals = [
            (
                datetime.strptime(entry["startDate"], "%Y-%m-%dT%H:%MZ"),
                datetime.strptime(entry["endDate"], "%Y-%m-%dT%H:%MZ")
            ) 
            for entry in data["leagues"][0]["calendar"][0]["entries"]
        ]

        final_date = datetime.strptime(final_date, "%Y%m%d")
        for start, end in intervals:
            current_date = start
            while current_date <= end and current_date <= final_date + timedelta(days=1):
                self.__get_date(current_date.strftime("%Y%m%d"))
                current_date += timedelta(days = 1)
    
    def __get_date(self, date):
        print(date)
        r = requests.get(f"{self.url}/{date}")
        soup = BeautifulSoup(r.text)
        
        for script in soup.find_all("script", {"type": None, "src": None}):
            if script.text.startswith("window.espn.scoreboardData"):
                data = json.loads(script.text[30:-337])
                break
        
        
        for event in data["events"]:
            comp = event["competitions"][0]
            team0_ha = comp["competitors"][0]["homeAway"]
            team1_ha = comp["competitors"][1]["homeAway"]
            dtt_utf0 = datetime.strptime(comp["date"], "%Y-%m-%dT%H:%MZ") + timedelta(hours=3)
            
            game = {
                "dtt": datetime.strftime(dtt_utf0, "%Y-%m-%dT%H:%M:%SZ"),
                team0_ha: comp["competitors"][0]["team"]["displayName"],
                team1_ha: comp["competitors"][1]["team"]["displayName"],
                f"{team0_ha}score": comp["competitors"][0]["score"],
                f"{team1_ha}score": comp["competitors"][1]["score"],
                "stadium": comp["venue"]["fullName"],
                "country": comp["venue"]["address"]["country"],
                "city": comp["venue"]["address"]["city"],
            }
            self.games.append(game)

In [2]:
conmebol = CONMEBOL_ESPN("20221029")

20220101
20220102
20220103
20220104
20220105
20220106
20220107
20220108
20220109
20220110
20220111
20220112
20220113
20220114
20220115
20220116
20220117
20220118
20220119
20220120
20220121
20220122
20220123
20220124
20220125
20220126
20220127
20220128
20220129
20220130
20220131
20220201
20220202
20220203
20220204
20220205
20220206
20220207
20220208
20220209
20220210
20220211
20220212
20220213
20220214
20220215
20220216
20220217
20220218
20220219
20220220
20220221
20220222
20220223
20220224
20220225
20220226
20220227
20220228
20220301
20220302
20220303
20220304
20220305
20220306
20220307
20220308
20220309
20220310
20220311
20220312
20220313
20220314
20220315
20220316
20220317
20220318
20220319
20220320
20220321
20220322
20220323
20220324
20220325
20220326
20220327
20220328
20220329
20220330
20220331
20220401
20220402
20220403
20220404
20220405
20220406
20220407
20220408
20220409
20220410
20220411
20220412
20220413
20220414
20220415
20220416
20220417
20220418
20220419
20220420
20220421
2

In [3]:
conmebol.games

[{'dtt': '2022-02-09T03:30:00Z',
  'home': 'Montevideo City Torque',
  'away': 'Barcelona SC',
  'homescore': '1',
  'awayscore': '1',
  'stadium': 'Estadio Centenario',
  'country': 'Uruguai',
  'city': 'Montevideo'},
 {'dtt': '2022-02-10T01:15:00Z',
  'home': 'Deportivo Lara',
  'away': 'Bolívar',
  'homescore': '2',
  'awayscore': '3',
  'stadium': 'Estadio Agustín Tovar',
  'country': 'Venezuela',
  'city': 'Barinas'},
 {'dtt': '2022-02-10T03:30:00Z',
  'home': 'César Vallejo',
  'away': 'Olimpia',
  'homescore': '0',
  'awayscore': '1',
  'stadium': 'Estadio Nacional de Lima',
  'country': 'Peru',
  'city': 'Lima'},
 {'dtt': '2022-02-16T03:30:00Z',
  'home': 'Barcelona SC',
  'away': 'Montevideo City Torque',
  'homescore': '0',
  'awayscore': '0',
  'stadium': 'Estadio Banco Pichincha',
  'country': 'Equador',
  'city': 'Guayaquil'},
 {'dtt': '2022-02-17T01:15:00Z',
  'home': 'Bolívar',
  'away': 'Deportivo Lara',
  'homescore': '4',
  'awayscore': '0',
  'stadium': 'Estadio Hern

In [5]:
with open("libertadores2022.json", "w+") as outfile:
    json.dump(conmebol.games, outfile)